In this step, we preprocess the raw voice recordings by combining all audio files within each recording folder, then converting them into mel spectrograms. These spectrograms serve as input for our convolutional neural network.


The final spectrograms are saved as `.png` images, organized by class (PD / NON PD).

In [ ]:
import os
import librosa
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import librosa.display
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

audio_exts = ('.wav', '.m4a', '.aac')

def process_record(args):
    patient_folder, record_folder, input_folder, output_folder = args

    record_path = os.path.join(input_folder, patient_folder, record_folder)
    voice_files = [f for f in os.listdir(record_path) if f.lower().endswith(audio_exts)]
    if len(voice_files) == 0:
        return f"Skipped (no audio): {record_path}"

    concatenated_audio = []
    sr = None

    for voice_file in voice_files:
        audio_path = os.path.join(record_path, voice_file)
        try:
            audio, current_sr = librosa.load(audio_path, sr=None)
            if sr is None:
                sr = current_sr
            elif sr != current_sr:
                return f"Skipped (inconsistent sampling rates): {record_path}"
            concatenated_audio.append(audio)
        except Exception as e:
            return f"Failed to load {audio_path}: {e}"

    if not concatenated_audio or sr is None or sr <= 0:
        return f"Skipped (no valid audio or sampling rate): {record_path}"

    concatenated_audio = np.concatenate(concatenated_audio)

    mel_spec = librosa.feature.melspectrogram(y=concatenated_audio, sr=sr)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    output_path = os.path.join(output_folder, patient_folder, record_folder)
    os.makedirs(output_path, exist_ok=True)

    img_path = os.path.join(output_path, 'spectrogram.png')
    plt.figure(figsize=(10, 4))
    plt.axis('off')
    librosa.display.specshow(mel_spec_db, sr=sr, x_axis='time', y_axis='mel')
    plt.tight_layout(pad=0)
    plt.savefig(img_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    return f"Saved: {img_path}"

def concat_and_convert_parallel(input_folder, output_folder):
    tasks = []

    for patient_folder in os.listdir(input_folder):
        patient_path = os.path.join(input_folder, patient_folder)
        if not os.path.isdir(patient_path):
            continue

        for record_folder in os.listdir(patient_path):
            record_path = os.path.join(patient_path, record_folder)
            if not os.path.isdir(record_path):
                continue

            tasks.append((patient_folder, record_folder, input_folder, output_folder))

    print(f"Processing {len(tasks)} record folders using {cpu_count()} CPU cores...")

    with Pool(processes=cpu_count()) as pool:
        for r in tqdm(pool.imap_unordered(process_record, tasks), total=len(tasks), desc=f"Processing {os.path.basename(input_folder)}"):
            print(r)

# Run for both folders
concat_and_convert_parallel("/content/dataset/non pd", "/content/dataset/NONPD")
concat_and_convert_parallel("/content/dataset/pd", "/content/dataset/PD")

Processing 54 record folders using 2 CPU cores...


Processing non pd:   2%|▏         | 1/54 [00:43<38:22, 43.43s/it]

Saved: /content/dataset/NONPD/patient 9 non pd/recording 3/spectrogram.png
Saved: /content/dataset/NONPD/patient 9 non pd/recording 1/spectrogram.png


Processing non pd:   6%|▌         | 3/54 [00:47<10:59, 12.92s/it]

Saved: /content/dataset/NONPD/patient 4 non pd/recording 1/spectrogram.png
Saved: /content/dataset/NONPD/patient 9 non pd/recording 2/spectrogram.png


...

Saved: /content/dataset/PD/patient 1 pd/recording 1/spectrogram.png


Processing pd:  98%|█████████▊| 39/40 [01:57<00:02,  2.92s/it]

Saved: /content/dataset/PD/patient 1 pd/recording 4/spectrogram.png


Processing pd: 100%|██████████| 40/40 [01:57<00:00,  2.95s/it]

Saved: /content/dataset/PD/patient 1 pd/recording 2/spectrogram.png
